# final version

In [1]:
# 📦 Imports
import cv2
import os
import numpy as np
from ultralytics import YOLO

# 🧠 Local utilities
from utils.tracker import ObjectTracker
from utils.speed_estimation import (
    calculate_speed,
    SpeedAggregator,
    speed_category,
    format_top_right_text
)

# 🔧 Configuration
base_dir = r"D:\Projects\ball_detection"
VIDEO_INPUT = os.path.join(base_dir, "vedios", "input", "example_clip3.mp4")
VIDEO_OUTPUT = os.path.join(base_dir, "vedios", "output", "processed_clip.mp4")

MODEL_PATH_BALL = os.path.join(base_dir, "runs", "cric_detect_train_bat_ball_sample2", "weights", "best.pt")
MODEL_PATH_BAT  = os.path.join(base_dir, "runs", "cric_detect_train_bat_ball_finetune3", "weights", "best.pt")

# If you have a known reference (e.g., crease-to-crease pixel distance), set it here
# Example: 20.12 m pitch length (stump-to-stump), measured_pixels = number of pixels covering that distance.
KNOWN_DISTANCE_METERS   = None   # e.g., 20.12
KNOWN_DISTANCE_PIXELS   = None   # e.g., 800
DEFAULT_PIXELS_PER_METER = 50.0  # Fallback if no reference calibration is provided

BALL_CLASS_ID = 0
BAT_CLASS_ID  = 1

# 🧠 Helper
def check_overlap(box1, box2):
    x1a, y1a, x2a, y2a = box1
    x1b, y1b, x2b, y2b = box2
    return not (x1a > x2b or x2a < x1b or y1a > y2b or y2a < y1b)

def compute_pixels_per_meter():
    if KNOWN_DISTANCE_METERS and KNOWN_DISTANCE_PIXELS and KNOWN_DISTANCE_METERS > 0 and KNOWN_DISTANCE_PIXELS > 0:
        return KNOWN_DISTANCE_PIXELS / KNOWN_DISTANCE_METERS
    return DEFAULT_PIXELS_PER_METER

# 🎥 Processing function
def process_cricket_video():
    print("🚀 Starting cricket video processing...")

    os.makedirs(os.path.dirname(VIDEO_OUTPUT), exist_ok=True)

    # Load models
    model_ball = YOLO(MODEL_PATH_BALL)
    model_bat  = YOLO(MODEL_PATH_BAT)
    print("✅ Models loaded.")

    # Load video
    cap = cv2.VideoCapture(VIDEO_INPUT)
    if not cap.isOpened():
        print(f"❌ Failed to open video: {VIDEO_INPUT}")
        return

    fps         = cap.get(cv2.CAP_PROP_FPS)
    width       = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height      = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames= int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"🎬 FPS={fps}, Resolution={width}x{height}, Frames={total_frames}")

    out = cv2.VideoWriter(VIDEO_OUTPUT, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    tracker = ObjectTracker(distance_threshold=120.0, iou_threshold=0.3, max_lost=8, frame_size=(width, height))

    ppm = compute_pixels_per_meter()  # pixels per meter for conversion
    speeds = SpeedAggregator(window=7, spike_kmph_cap=250.0, min_kmph_threshold=5.0)

    frame_count   = 0
    hit_occurred  = False
    last_ball_id  = None  # track the primary ball ID (most stable one)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1

        # Detect with both models
        results_ball = model_ball(frame, conf=0.25, iou=0.6, verbose=False)
        results_bat  = model_bat(frame,  conf=0.25, iou=0.6, verbose=False)

        ball_dets, bat_dets = [], []

        # Ball detections
        if results_ball and len(results_ball[0].boxes) > 0:
            boxes   = results_ball[0].boxes.xyxy.cpu().numpy()
            classes = results_ball[0].boxes.cls.cpu().numpy()
            confs   = results_ball[0].boxes.conf.cpu().numpy()
            for box, cls, cf in zip(boxes, classes, confs):
                if int(cls) == BALL_CLASS_ID:
                    x1, y1, x2, y2 = map(int, box)
                    # optional: tighten tiny boxes or skip absurdly large boxes
                    if 2 <= (x2 - x1) <= width and 2 <= (y2 - y1) <= height:
                        ball_dets.append((x1, y1, x2, y2))

        # Bat detections
        if results_bat and len(results_bat[0].boxes) > 0:
            boxes   = results_bat[0].boxes.xyxy.cpu().numpy()
            classes = results_bat[0].boxes.cls.cpu().numpy()
            for box, cls in zip(boxes, classes):
                if int(cls) == BAT_CLASS_ID:
                    x1, y1, x2, y2 = map(int, box)
                    bat_dets.append((x1, y1, x2, y2))
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(frame, "Bat", (x1, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255, 0, 0), 2)

        # Track ball
        tracked_objects = tracker.update(ball_dets)

        # Choose a primary ball id: prefer the longest-lived / lowest id present
        if tracked_objects:
            if last_ball_id not in tracked_objects:
                last_ball_id = min(tracked_objects.keys())

        # Process tracked ball objects
        for object_id, obj_data in tracked_objects.items():
            bbox     = obj_data['bbox']
            centroid = obj_data['centroid']
            x1, y1, x2, y2 = bbox
            cx, cy = centroid

            # Draw ball box/centroid
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 220, 0), 2)
            cv2.circle(frame, (cx, cy), 4, (0, 220, 0), -1)
            cv2.putText(frame, f"ID {object_id}", (x1, y1 - 18), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0, 255, 255), 2)

            # Speed updates only before hit
            if not hit_occurred:
                prev_c = speeds.get_prev_centroid(object_id)
                if prev_c is not None and fps > 0 and ppm > 0:
                    inst_kmph = calculate_speed(prev_c, centroid, fps, ppm)
                    # Add to aggregator with smoothing and spike checks
                    speeds.update(object_id, centroid, inst_kmph)
                    # Per-object instantaneous display (optional, can disable)
                    if inst_kmph > 0:
                        cv2.putText(frame, f"{inst_kmph:.1f} km/h", (x1, y1 - 2),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0, 0, 255), 2)
                else:
                    speeds.update(object_id, centroid, 0.0)
            else:
                # After hit, keep centroid history but do not update speed
                speeds.update(object_id, centroid, 0.0)

            # Hit detection: overlap of ball bbox with any bat bbox
            if not hit_occurred and bat_dets:
                is_hit = any(check_overlap(bbox, bat_bbox) for bat_bbox in bat_dets)
                if is_hit:
                    cv2.putText(frame, "HIT!", (width // 2 - 40, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
                    hit_occurred = True
                    speeds.freeze_on_hit()  # freeze updates and lock "at hit" value

        # Build top-right overlay: max speed always; after hit also show "at hit" based on max
        max_kmph = speeds.max_speed()
        at_hit_kmph = speeds.hit_speed()  # may be None if no hit yet
        cat = speed_category(max_kmph)

        # Compose two lines: primary and secondary (if hit)
        lines = []
        lines.append(f"Max Ball Speed: {max_kmph:.1f} km/h [{cat}]")
        if at_hit_kmph is not None:
            lines.append(f"Ball Speed at Hit: {at_hit_kmph:.1f} km/h")

        # Draw at top-right
        format_top_right_text(frame, lines, (width, height), pad_right=20, top_y=36, font_scale=0.9, thickness=2, color=(0, 255, 255))

        # Write frame
        out.write(frame)
        print(f"Processing frame {frame_count}/{total_frames}", end='\r')

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"\n✅ Done. Output saved to: {VIDEO_OUTPUT}")

In [2]:
process_cricket_video()

🚀 Starting cricket video processing...
✅ Models loaded.
🎬 FPS=60.0, Resolution=1920x1080, Frames=2227
Processing frame 2222/2227
✅ Done. Output saved to: D:\Projects\ball_detection\vedios\output\processed_clip.mp4
